# Imports

In [2]:
# %% Imports y configuración
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
from pathlib import Path
import json

# Agregar el directorio raíz al path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), '..'))

# Imports del proyecto
from model_ddp.utils.sistem_fun import (
    load_config,
    get_data_path,
    get_artifact_path,
    get_report_path,
    create_experiment_id,
    ensure_directories,
    save_experiment_metadata
)

from model_ddp.simulations.gaussian_simulator import (
    SimulationConfig,
    RBFKernel,
    MaternKernel,
    PeriodicKernel,
    LinearKernel,
    GaussianProcess,
    RegressionSimulator,
    TransformationFunctions
)

# Modelos
from model_ddp.models.LSBP_laplace_v1 import LSBPLaplace
from model_ddp.models.LSBP_normal_v3 import LSBPNormal

# Modelos Random forest y Xgboost 
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Metricas y graficas 
from model_ddp.fit.metrics import regression_metrics
from model_ddp.graphics.plots_regression import plot_regression_analysis
from model_ddp.graphics.plots_traces import plot_hyperparameter_traces
from model_ddp.graphics.plots_aplication import plot_credible_intervals

# Modulo pipeline
from model_ddp.pipelines.data_separacion import split_data

config=load_config()

# Configuracion Experimento 1

In [ ]:
#Parametros Iniciales
NOMBRE_EJECUCION = "comparacion_modelos_002"
SIM_REAL = "simulation"

In [ ]:
# Parámetros de ejecución de experimentos 
CARACTERISTICAS = "Comparacion modelos relaciones complejas con altas varianzas y heterogeneidad, observaciones 500"
EXPERIMENT_ID = create_experiment_id("comparacion_modelos_002")

In [11]:
##################################################
# Registrar Experimento
##################################################

# Preparar información del experimento
experiment_data = {
    'experiment_id': EXPERIMENT_ID,
    'nombre': NOMBRE_EJECUCION,
    'tipo': SIM_REAL,
    'descripcion': f"""Experimento: {CARACTERISTICAS}"""
}
registry_file = save_experiment_metadata(config, experiment_data)
print(f"✓ Experimento registrado en: {registry_file}")


✓ Experimento registrado en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\versioning\experiment_registry.md


In [ ]:
##################################################
# Crear carpeta de guardado 
##################################################
data_path = get_data_path(config, SIM_REAL, "output")
carpeta_datos = data_path / f"{EXPERIMENT_ID}"
carpeta_datos.mkdir(parents=True, exist_ok=True)

##################################################
# Configuración de la simulación
##################################################
sim_config = SimulationConfig(
    n_samples=600,
    n_features=8,
    x_range=(0.0, 2.0),
    noise_std=1.0,              # ruido base (se sobreescribe por régimen)
    random_state=324
)

##################################################
# Kernel para generar X
##################################################
kernel = RBFKernel(
    length_scale=2.0,
    variance=1.0
)

##################################################
# Definición de regímenes latentes
##################################################
rng = np.random.RandomState(324)

n_components = 3
mixture_weights = np.array([0.4, 0.35, 0.25])

# Funciones por régimen
transformations = [
    TransformationFunctions.polynomial(degree=2),
    TransformationFunctions.polynomial(degree=3),
    TransformationFunctions.nonlinear_combination()
]

# Varianzas por régimen (heterocedasticidad)
noise_stds = [0.5, 2.0, 4.0]

##################################################
# Crear simulador base (solo para X)
##################################################
simulator = RegressionSimulator(
    config=sim_config,
    kernel=kernel,
    transformation=lambda X: np.zeros(X.shape[0])  # placeholder
)

##################################################
# Generar datos
##################################################
print("Generando datos (escenario 2: heterogeneidad + ruido)...")

X = simulator.generate_covariates()

# Asignación latente de regímenes
z = rng.choice(n_components, size=sim_config.n_samples, p=mixture_weights)

# Generar Y heterogéneo
Y = np.zeros(sim_config.n_samples)

for k in range(n_components):
    idx = z == k
    Y_clean = transformations[k](X[idx])
    noise = rng.normal(0, noise_stds[k], size=Y_clean.shape)
    Y[idx] = Y_clean + noise

print("✓ Datos generados exitosamente")

##################################################
# Estadísticas
##################################################
print(f"\nEstadísticas de X:")
print(f"  Shape: {X.shape}")
print(f"  Media por feature: {X.mean(axis=0)}")
print(f"  Std por feature: {X.std(axis=0)}")

print(f"\nEstadísticas de Y:")
print(f"  Shape: {Y.shape}")
print(f"  Media: {Y.mean():.4f}")
print(f"  Std: {Y.std():.4f}")
print(f"  Min: {Y.min():.4f}")
print(f"  Max: {Y.max():.4f}")

##################################################
# Transformar a DataFrame
##################################################
datos = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(sim_config.n_features)])
datos['Y'] = Y


Generando datos...
✓ Datos generados exitosamente

Estadísticas de X:
  Shape: (600, 8)
  Media por feature: [-0.33127803  0.18296413  0.08537421 -1.33944282  0.34900027 -0.20726501
  0.31669675 -0.48512796]
  Std por feature: [0.20227712 0.02968197 0.25380141 0.48419649 0.55419153 0.26362322
 0.05877079 0.04867435]

Estadísticas de Y:
  Shape: (600,)
  Media: -3.0836
  Std: 3.2244
  Min: -13.4633
  Max: 5.6204


In [54]:
##################################################
# Separar data
##################################################
splits = split_data(
    X=X,
    y=Y,
    test_size=0.2,
    val_size=None,      #None si no quieres validación
    random_state=123
)

# Nombres de columnas
feature_cols = [f'X{i+1}' for i in range(X.shape[1])]

# Train
data_train = pd.DataFrame(splits["X_train"], columns=feature_cols)
data_train["Y"] = splits["y_train"]

# Test
data_test = pd.DataFrame(splits["X_test"], columns=feature_cols)
data_test["Y"] = splits["y_test"]

##################################################
# Guardar data frame  
##################################################
csv_filename = f"{carpeta_datos}/data_train.csv"
data_train.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

csv_filename = f"{carpeta_datos}/data_test.csv"
data_test.to_csv(csv_filename, index=False)

print(f"✓ Datos guardados en CSV: {csv_filename}")

✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\comparacion_modelos_001_20251225_214401/data_train.csv
✓ Datos guardados en CSV: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\data\simulaciones\comparacion_modelos_001_20251225_214401/data_test.csv


## Modelos

### Modelo LSBP Kernel Normal (TRAIN)

In [59]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPNormal...")
print("="*60)

# Crear instancia del modelo
lsbp_model_normal = LSBPNormal(
    y=data_train["Y"].values,
    X=data_train.drop(columns=["Y"]).values,
    H=20,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = lsbp_model_normal.run(
    iterations=2000,          # Iteraciones totales
    burnin=500               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPNormal...
Using C++ acceleration
Iter 100/2000: K_eff=28, H=100, μ=0.99, μ₀=0.10, κ₀=0.25, a₀=20.00, b₀=2.94
  Acceptance: α=0.94, ψ=0.87, κ=0.79, a=0.49
Iter 200/2000: K_eff=38, H=100, μ=1.77, μ₀=0.27, κ₀=0.40, a₀=20.00, b₀=5.05
  Acceptance: α=0.81, ψ=0.81, κ=0.67, a=0.40
Iter 300/2000: K_eff=29, H=100, μ=1.93, μ₀=0.22, κ₀=0.31, a₀=20.00, b₀=4.04
  Acceptance: α=0.77, ψ=0.70, κ=0.65, a=0.47
Iter 400/2000: K_eff=20, H=100, μ=2.96, μ₀=0.32, κ₀=0.28, a₀=20.00, b₀=4.00
  Acceptance: α=0.75, ψ=0.69, κ=0.51, a=0.53
Iter 500/2000: K_eff=24, H=100, μ=3.23, μ₀=0.49, κ₀=0.20, a₀=20.00, b₀=3.32
  Acceptance: α=0.59, ψ=0.58, κ=0.49, a=0.46
Iter 600/2000: K_eff=42, H=100, μ=2.90, μ₀=0.42, κ₀=0.44, a₀=20.00, b₀=3.79
  Acceptance: α=0.69, ψ=0.54, κ=0.35, a=0.51
Iter 700/2000: K_eff=30, H=100, μ=2.71, μ₀=0.16, κ₀=0.34, a₀=20.00, b₀=2.44
  Acceptance: α=0.53, ψ=0.54, κ=0.30, a=0.51
Iter 800/2000: K_eff=38, H=100, μ=2.15, μ₀=0.36, κ₀=0.40, a₀=20.00, b₀=3.21
  Acceptance: α=0.62, ψ=0.51

### Modelo LSBP Kernel Laplace (Train)

In [60]:
##################################################
# Modelo   
##################################################
print("\n" + "="*60)
print("EJECUTANDO LSBPLaplace...")
print("="*60)

# Crear instancia del modelo
lsbp_model_laplace = LSBPLaplace(
    y=datos["Y"].values,
    X=datos.drop(columns=["Y"]).values,
    H=20,                     # Número inicial de clusters truncados
    verbose=True              # Mostrar progreso
)

# Ejecutar MCMC
trace = lsbp_model_laplace.run(
    iterations=2000,          # Iteraciones totales
    burnin=500               # Burn-in
)

print("\n" + "="*60)
print("LSBP COMPLETADO")
print("="*60)


EJECUTANDO LSBPLaplace...
Using C++ acceleration for 8 functions (compute_eta, compute_weights, update_lambda_latent, update_assignments, update_atoms, update_alpha, update_psi, update_ell)
Iter 100/2000: K_eff=85, H=100, μ=0.27, μ₀=0.03, τ₀=0.65, a₀=0.52, β₀=12.61
  Acceptance: α=0.79, ψ=0.76, τ=0.57, a=0.61
Iter 200/2000: K_eff=88, H=100, μ=0.33, μ₀=-0.15, τ₀=0.73, a₀=0.65, β₀=23.01
  Acceptance: α=0.69, ψ=0.75, τ=0.55, a=0.48
Iter 300/2000: K_eff=83, H=100, μ=0.56, μ₀=-0.13, τ₀=0.99, a₀=0.62, β₀=30.71
  Acceptance: α=0.67, ψ=0.62, τ=0.38, a=0.31
Iter 400/2000: K_eff=85, H=100, μ=0.39, μ₀=0.20, τ₀=0.66, a₀=0.82, β₀=35.32
  Acceptance: α=0.65, ψ=0.51, τ=0.46, a=0.33
Iter 500/2000: K_eff=84, H=100, μ=0.55, μ₀=0.01, τ₀=0.47, a₀=0.77, β₀=32.07
  Acceptance: α=0.51, ψ=0.55, τ=0.39, a=0.28
Iter 600/2000: K_eff=88, H=100, μ=0.58, μ₀=-0.00, τ₀=0.61, a₀=0.83, β₀=37.50
  Acceptance: α=0.54, ψ=0.49, τ=0.34, a=0.37
Iter 700/2000: K_eff=89, H=100, μ=0.74, μ₀=-0.11, τ₀=0.64, a₀=0.71, β₀=33.84
  A

### Modelo Random Forest (Train)

In [55]:
# Separar features y target
X_train = data_train[feature_cols]  # DataFrame con nombres
y_train = data_train["Y"]

X_test = data_test[feature_cols]
y_test = data_test["Y"]

rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=123,
    n_jobs=-1,
    verbose=1   
)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.8s finished


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",500
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",5
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",3
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples 

### Modelo Xgboost (Train)

In [56]:
# Separar features y target
X_train = data_train[feature_cols]
y_train = data_train["Y"]

X_test = data_test[feature_cols]
y_test = data_test["Y"]

# Crear el modelo
xgb_model = XGBRegressor(
    n_estimators=500,        # Número de árboles
    max_depth=6,             # Profundidad máxima de cada árbol
    learning_rate=0.1,       # Tasa de aprendizaje (shrinkage)
    subsample=0.8,           # Fracción de muestras para cada árbol
    colsample_bytree=0.8,    # Fracción de features para cada árbol
    min_child_weight=3,      # Similar a min_samples_leaf
    gamma=0,                 # Regularización por reducción mínima de pérdida
    reg_alpha=0,             # L1 regularization
    reg_lambda=1,            # L2 regularization
    random_state=123,
    n_jobs=-1,
    verbosity=1              # Equivalente a verbose
)

# Entrenar el modelo
xgb_model.fit(X_train, y_train)

,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.8
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes 

## Evaluamos 

### LSBP Normal

In [61]:
########################################################
# Hacer predicciones con el modelo entrenado
########################################################
y_pred_mean, y_pred_std = lsbp_model_normal.predict_mean(
    X_new=data_test.drop(columns=["Y"]).values,
    n_samples=1000
)
y_true = data_test["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics_lsbp_model_normal.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

########################################################
# Predicciones completas para generar graficas
########################################################
# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})

##################################################
# Gráficas de Fit 
##################################################

# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}_LSBP_Normal"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Test Set")
]
plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="LSBP_Normal"
)
print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   3.302534
  RMSE    :   1.817288
  MAE     :   1.425259
  R2      :   0.627205
  MAPE    :  84.739033
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401\metrics_lsbp_model_normal.json
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401_LSBP_Normal


### LSBP Laplace

In [62]:
########################################################
# Hacer predicciones con el modelo entrenado
########################################################
y_pred_mean, y_pred_std = lsbp_model_laplace.predict_mean(
    X_new=data_test.drop(columns=["Y"]).values,
    n_samples=1000
)
y_true = data_test["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred_mean)

print("\n📊 MÉTRICAS DE AJUSTE:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics_lsbp_model_laplace.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

########################################################
# Predicciones completas para generar graficas
########################################################
# Guardar predicciones completas
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred_mean,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred_mean,
    'residual_std': (y_true - y_pred_mean) / y_pred_std  # Residuos estandarizados
})

##################################################
# Gráficas de Fit 
##################################################
# Crear carpeta para gráficas
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}_LSBP_Laplace"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred_mean, "Test Set")
]
plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="LSBP_Laplace"
)
print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


📊 MÉTRICAS DE AJUSTE:
------------------------------------------------------------
  MSE     :   3.284935
  RMSE    :   1.812439
  MAE     :   1.445115
  R2      :   0.629191
  MAPE    :  88.061421
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401\metrics_lsbp_model_laplace.json
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401_LSBP_Laplace


### Random Forest

In [57]:
########################################################
# Hacer predicciones con Random Forest
########################################################

# Predicciones
y_pred = rf_model.predict(X_test)
y_true = y_test.values               # Convertimos a numpy array para consistencia

# Calcular métricas
metrics = regression_metrics(y_true, y_pred)

print("\n📊 MÉTRICAS DE AJUSTE RANDOM FOREST:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta para guardar reportes
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics_rf_model.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

########################################################
# Predicciones completas para gráficas
########################################################
# Desviación estándar de los árboles individuales
all_tree_preds = pd.DataFrame([tree.predict(X_test) for tree in rf_model.estimators_])
y_pred_std = all_tree_preds.std(axis=0).values

predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred,
    'y_pred_std': y_pred_std,
    'residual': y_true - y_pred,
    'residual_std': (y_true - y_pred) / (y_pred_std + 1e-8)  # Evitar división por 0
})

##################################################
# Gráficas de Fit 
##################################################
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}_RandomForest"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred, "Test Set")
]
plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="RandomForest"
)
print(f"✓ Gráficas guardadas en: {carpeta_graficas}")


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.3s finished
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-


📊 MÉTRICAS DE AJUSTE RANDOM FOREST:
------------------------------------------------------------
  MSE     :   4.340332
  RMSE    :   2.083346
  MAE     :   1.689636
  R2      :   0.510056
  MAPE    : 166.653011
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401\metrics_rf_model.json


c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
c:\Users\JuanFran\anaconda3\envs\model_ddp\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without featur

✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401_RandomForest


### XGboost

In [58]:
########################################################
# Hacer predicciones con XGBoost
########################################################
y_pred = xgb_model.predict(data_test[feature_cols].values)
y_true = data_test["Y"].values

# Calcular métricas
metrics = regression_metrics(y_true, y_pred)

print("\n📊 MÉTRICAS DE AJUSTE XGBOOST:")
print("-" * 60)
for metric_name, metric_value in metrics.items():
    print(f"  {metric_name.upper():8s}: {metric_value:10.6f}")
print("-" * 60)

# Carpeta para guardar reportes
report_path = get_report_path(config, SIM_REAL, "tables")
carpeta_reportes = report_path / f"{EXPERIMENT_ID}"
carpeta_reportes.mkdir(parents=True, exist_ok=True)

# Guardar métricas en JSON
metrics_file = carpeta_reportes / "metrics_xgb_model.json"
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"\n✓ Métricas guardadas: {metrics_file}")

########################################################
# Predicciones completas para gráficas
########################################################
# Para XGBoost no tenemos std directamente, se puede poner NaN o 0
predictions_df = pd.DataFrame({
    'y_true': y_true,
    'y_pred_mean': y_pred,
    'y_pred_std': 0,                # XGBoost no entrega std directamente
    'residual': y_true - y_pred,
    'residual_std': 0                # O calcular Z-score si quieres
})

##################################################
# Gráficas de Fit 
##################################################
graphics_path = get_report_path(config, SIM_REAL, "graphics")
carpeta_graficas = graphics_path / f"{EXPERIMENT_ID}_XGBoost"
carpeta_graficas.mkdir(parents=True, exist_ok=True)

# Generar gráficas usando el módulo
splits = [
    (y_true, y_pred, "Test Set")
]
plot_regression_analysis(
    splits=splits,
    output_path=str(carpeta_graficas),
    model_name="XGBoost"
)
print(f"✓ Gráficas guardadas en: {carpeta_graficas}")



📊 MÉTRICAS DE AJUSTE XGBOOST:
------------------------------------------------------------
  MSE     :   5.000099
  RMSE    :   2.236090
  MAE     :   1.747925
  R2      :   0.435581
  MAPE    : 160.667228
------------------------------------------------------------

✓ Métricas guardadas: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401\metrics_xgb_model.json
✓ Gráficas guardadas en: C:\Users\JuanFran\Desktop\git_tesis\model_ddp\reports\simulaciones\comparacion_modelos_001_20251225_214401_XGBoost
